In [ ]:
#Importing the required libraries

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from sklearn.metrics import f1_score

In [ ]:
!unzip Data.zip

In [ ]:
data_dir = "300ImageData/train"
classes = os.listdir(data_dir)

In [ ]:
#mounting google drive in colab to access the dataset from google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = "/content/drive/MyDrive/300ImageData"
classes = os.listdir(data_dir)

In [ ]:
# imageTransform. A function that returns a transformed image dataset .

def imageTransform(path):
  resize = transforms.Resize(size=(224,224))
  transforming = transforms.Compose([resize, transforms.ToTensor(), transforms.Normalize(0.5, 0.25)])
  dataTransform = ImageFolder(path, transforming)
  return dataTransform


In [ ]:
dataset = imageTransform(data_dir)

In [ ]:
random_seed = 42
torch.manual_seed(random_seed);

In [ ]:
#Generating the train and validation dataset

val_size = round(0.3 * len(dataset))
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(3884, 1664)

In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size=10

In [ ]:
# DataLoader for train and validation dataset

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import datetime

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    for name, loader in [("train", train_loader), ("val", val_loader)]:
      correct = 0
      total = 0

      with torch.no_grad():
        for imgs, labels in loader:
          outputs = model(imgs)
          _, predicted = torch.max(outputs, dim=1)
          total += labels.shape[0]
          correct += int((predicted == labels).sum())
    if epoch == 1 or epoch % 2 == 0:
      print('{} Epoch {}, Training loss {}, Accuracy {}: {:.2f}'.format(datetime.datetime.now(), epoch,
      loss_train / len(train_loader), name , correct / total))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

device(type='cuda')

In [ ]:
class Identity(nn.Module):
  def __init__(self):
    super(Identity, self).__init__()

  def forward(self, x):
    return x

In [ ]:
model = models.alexnet(pretrained=True)
print(model)

In [ ]:
model.classifier[6] = nn.Linear(4096, 4)

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device);

In [ ]:
optimize = optim.SGD(model.parameters(), lr=1e-2)
loss = nn.CrossEntropyLoss()

In [ ]:
model.eval()

In [ ]:
training_loop(20, optimize, model, loss, train_dl, val_dl)

In [ ]:
def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0

    with torch.no_grad():
      for imgs, labels in loader:
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    print("Accuracy {}: {:.2f}".format(name , correct / total))
    

In [ ]:
validate(model, train_dl, val_dl)

Accuracy train: 1.00
Accuracy val: 0.91


In [ ]:
for param in model.parameters():
  print(param)

In [ ]:
FILE = "model4.pth"

In [ ]:
torch.save(model.state_dict(), FILE)

In [ ]:
"""model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False"""

""" from zipfile import ZipFile
file_name = 'ImageData.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')"""